<a href="https://colab.research.google.com/github/KimaniKibuthu/Recommender-system-using-sentiment-analysis/blob/main/Amazon_Reviews_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Reviews Data Analysis
This notebook contains the work of using amazon data and using it for sentiment analysis which would later be used in a recommender system.



# Libraries and Functions

In [1]:
!pip install transformers

In [2]:
# Import necessary libraries
import re
import sys
import random
import gzip
import json
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import class_weight


import tensorflow as tf
#import tensorflow_text
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

from wordcloud import WordCloud, STOPWORDS
#import ktrain
#from ktrain import text
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

#import nlpaug.augmenter.word as naw

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Load Data

In [3]:
# Import data functions
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')



In [4]:
!wget -P /content/ http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Video_Games.json.gz

--2022-04-05 13:29:16--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Video_Games.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 522823613 (499M) [application/octet-stream]
Saving to: ‘/content/Video_Games.json.gz.1’

Video_Games.json.gz 100%[===================>] 498.60M  53.1MB/s    in 9.4s    

2022-04-05 13:29:26 (52.9 MB/s) - ‘/content/Video_Games.json.gz.1’ saved [522823613/522823613]



In [5]:
# Read data
games_reviews = getDF('/content/Video_Games.json.gz')


In [6]:
games_reviews.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN
1,3.0,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN
2,4.0,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15,NaN,NaN
3,1.0,True,"02 7, 2014",A2XO1JFCNEYV3T,0439381673,ColoradoPartyof5,The product description should state this clea...,does not work on Mac OSX,1391731200,11,NaN,NaN
4,4.0,True,"01 16, 2014",A19WLPIRHD15TH,0439381673,Karen Robinson,I would recommend this learning game for anyon...,Roughing it,1389830400,NaN,NaN,NaN


# Data Cleaning and Preprocessing

## Subset Data
Obtain the features necessary.

In [7]:
# Get the columns
games_reviews.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style', 'image'],
      dtype='object')

In [8]:
# Features to use
features_to_use = ['overall', 'verified',
                   'reviewerID', 'asin',
                   'reviewText', 'summary',
                   'vote']
          

In [9]:
# Subset the data
games_subset = games_reviews[features_to_use]

## Clean the Data

In [10]:
# Check summary
games_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2565349 entries, 0 to 2565348
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   overall     float64
 1   verified    bool   
 2   reviewerID  object 
 3   asin        object 
 4   reviewText  object 
 5   summary     object 
 6   vote        object 
dtypes: bool(1), float64(1), object(5)
memory usage: 139.5+ MB


In [11]:

# Check for null values
games_subset.isnull().sum()

overall             0
verified            0
reviewerID          0
asin                0
reviewText       1715
summary           811
vote          2122633
dtype: int64

In [12]:
# Dropna for text and summary
games_subset.dropna(subset=['reviewText', 'summary'], how='any', inplace=True)

# Fill votes with zero
games_subset['vote'].fillna(0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [13]:
# Drop duplicates
games_subset.duplicated().sum()

74345

In [14]:
# Drop duplicates
games_subset.drop_duplicates(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [15]:
# View the data
games_subset.head()

,overall,verified,reviewerID,asin,reviewText,summary,vote
0,1.0,True,A21ROB4YDOZA5P,0439381673,I used to play this game years ago and loved i...,Did not like this,0
1,3.0,True,A3TNZ2Q5E7HTHD,0439381673,The game itself worked great but the story lin...,Almost Perfect,0
2,4.0,True,A1OKRM3QFEATQO,0439381673,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,15
3,1.0,True,A2XO1JFCNEYV3T,0439381673,The product description should state this clea...,does not work on Mac OSX,11
4,4.0,True,A19WLPIRHD15TH,0439381673,I would recommend this learning game for anyon...,Roughing it,0


In [16]:
# Turn overall to int 
games_subset['overall'] = games_subset['overall'].astype('int')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
games_subset['classes'] = games_subset['overall'] - 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
games_subset.head()

,overall,verified,reviewerID,asin,reviewText,summary,vote,classes
0,1,True,A21ROB4YDOZA5P,0439381673,I used to play this game years ago and loved i...,Did not like this,0,0
1,3,True,A3TNZ2Q5E7HTHD,0439381673,The game itself worked great but the story lin...,Almost Perfect,0,2
2,4,True,A1OKRM3QFEATQO,0439381673,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,15,3
3,1,True,A2XO1JFCNEYV3T,0439381673,The product description should state this clea...,does not work on Mac OSX,11,0
4,4,True,A19WLPIRHD15TH,0439381673,I would recommend this learning game for anyon...,Roughing it,0,3


In [19]:
games_subset_sample = games_subset.sample(n=30000, random_state=7, ignore_index=True)

## Prepping for NLP

In [20]:
# Subset for NLP
games_texts = games_subset_sample['reviewText'].values
games_summaries = games_subset_sample['summary'].values
games_ratings = games_subset_sample['classes'].values

In [21]:
# Text Cleaner 
def bert_preprocess_text(texts):
    """
  Clean texts and readies them for bert preprocessing

  Args:
    texts (list): A list containing texts to be cleaned
  
  Returns:
    new_texts (list):A list containing cleaned text

  """
    new_texts = []
    for word in texts:
        # Make lower
        word = word.lower()

        # Remove url
        word = re.sub(r'https?:\/\/\S+', '', word)
        word = re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', word)

        # Remove punctuation
        word = re.sub(r'[^\w\s]', '', word)

        # Remove placeholders
        word = re.sub(r"\[video\]", '', word)
        word = re.sub(r'{link}', '', word)

        # HTML and non letter characters
        word = re.sub(r'&[a-z]+;', '', word)
        word = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', word)

        # Remove mentions
        word = re.sub('@[\w]+', '', word)

        # lemmatize and remove stop words
        tokens = word_tokenize(word)
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]
        word = ' '.join(tokens)

        # Append clean texts to new_texts
        new_texts.append(word)

    # Tokenize texts

    return new_texts

# Experiment 1: With Review Texts

## Clean the Texts

In [22]:
# View distribution of ratings
unique, counts = np.unique(games_ratings, return_counts=True)
combined = list(zip(unique, counts/sum(counts)))
combined

[(0, 0.12493333333333333),
 (1, 0.05446666666666666),
 (2, 0.0838),
 (3, 0.16133333333333333),
 (4, 0.5754666666666667)]

In [23]:
# There is imbalance
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(games_ratings),
                                                 y=games_ratings)

In [24]:
class_weights

array([1.60085379, 3.67197062, 2.38663484, 1.23966942, 0.34754402])

In [25]:
classes = np.unique(games_ratings)

class_weight_dict = {}
for i in zip(classes, class_weights):
  class_weight_dict[i[0]] = i[1]



In [26]:
class_weight_dict

{0: 1.6008537886873,
 1: 3.6719706242350063,
 2: 2.386634844868735,
 3: 1.2396694214876034,
 4: 0.34754402224281744}

In [27]:
new_review_texts = bert_preprocess_text(games_texts)

In [28]:
# Split data
training_sentences, prel_sentences, training_labels, prel_labels = train_test_split(new_review_texts, 
                                                                                    games_ratings,
                                                                                    test_size=.1,
                                                                                    stratify = games_ratings)

test_sentences, validation_sentences, test_labels, validation_labels = train_test_split(prel_sentences, 
                                                                                        prel_labels,
                                                                                        test_size=.5,
                                                                                        stratify = prel_labels)


In [29]:
# Get BERT Tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [30]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True
                            )
test_encodings = tokenizer(test_sentences,
                            truncation=True,
                            padding=True
                          )


In [31]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
auc = tf.keras.metrics.AUC(name='auc', from_logits=True)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True,
                                                   name='sparse_categorical_crossentropy' )
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                 patience=3)

  # reduce learning rate
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                  factor = 0.1,
                                  patience = 2,
                                  min_lr = 1e-6,
                                  mode = 'min',
                                  verbose = 1)

callbacks = [early_stopper, reduce_lr]




In [ ]:
model.summary()

In [ ]:

history = model.fit(train_dataset.shuffle(len(training_sentences)).batch(16),
          epochs=10,
          batch_size=16,
          callbacks = callbacks,
          validation_data = val_dataset.shuffle(len(validation_sentences)).batch(16),
          class_weight = class_weight_dict
          )